# Concatenar las Bases de datos por Provincias

In [1]:
import pandas as pd
from os import listdir
import unidecode

In [2]:
additional_info = pd.read_csv(r'C:\Users\hecto\github_folder\spa_housing_crawler\additional_csv\addinfo.csv')
renta_pv = pd.read_csv(r'C:\Users\hecto\github_folder\spa_housing_crawler\additional_csv\rentas_PV.csv',  encoding='ISO-8859-1', sep=';')
rentas_espanya = pd.read_csv(r'C:\Users\hecto\github_folder\spa_housing_crawler\additional_csv\rentas_espanya.csv').rename(columns={'Unnamed: 0':'provincia'})

In [3]:
# Remove code, accencts and all names to lower case
additional_info['poblacion'] = additional_info['poblacion'].apply(lambda x: unidecode.unidecode(' '.join(x.split(' ')[1:])).lower())

rentas_espanya['provincia'] = rentas_espanya['provincia'].apply(lambda x: unidecode.unidecode(''.join(x.split('-')[0])).lower())

renta_pv['ambitos territoriales'] = renta_pv['ambitos territoriales'].apply(lambda x: unidecode.unidecode(x).lower())

In [4]:
all_files = []
#path = 'province_houses/'
path= r'C:\Users\hecto\github_folder\spa_housing_crawler\province_houses/'
for filename in listdir(path):
    if 'houses' in filename and 'csv' in filename:
        provincia = unidecode.unidecode(' '.join(filename.split('_')[1:len(filename.split('_')):1]).split('.')[0])
        renta_media = rentas_espanya[rentas_espanya['provincia'].str.contains(provincia)]['Renta disponible media']
        if renta_media.empty:
            renta_media = renta_pv[(renta_pv['ambitos territoriales'].str.contains(provincia)) & (renta_pv['tipo de renta'] == 'Renta total')]['2016']
        if renta_media.empty:
            renta_media = None
        else:
            renta_media = renta_media.values[0]
            
        valor_empresa_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(provincia)) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_muni = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains(provincia)) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))
        
        valor_empresa_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Empresas')]['valor'].values[0].replace('.', ''))
        valor_poblacion_nacional = pd.to_numeric(additional_info[(additional_info['poblacion'].str.contains('nacional')) & (additional_info['source'] == 'Poblacion')]['valor'].values[0].replace('.', ''))

        
        df = pd.read_csv(path + filename)
        df['source'] = filename.split('.')[0]
        df['number_of_companies_prov'] = valor_empresa_muni
        df['population_prov'] = valor_poblacion_muni
        df['companies_prov_vs_national_%'] = ((valor_empresa_muni / valor_empresa_nacional) * 100).round(2)
        df['population_prov_vs_national_%'] = ((valor_poblacion_muni / valor_poblacion_nacional) * 100).round(2)
        df['renta_media_prov'] = renta_media

        
        all_files.append(df)

In [5]:
result = pd.concat(all_files)

In [6]:
result.ground_size.unique()

array([nan, 'ground_size'], dtype=object)

In [7]:
df_comprobar=result[['loc_zone','source']]

In [8]:
df_comprobar=df_comprobar.reset_index()
df_comprobar

,index,loc_zone,source
0,0,"Zuya, Álava",houses_alava
1,1,"Zuya, Álava",houses_alava
2,2,"Zuya, Álava",houses_alava
3,3,"Zuya, Álava",houses_alava
4,4,"Zuya, Álava",houses_alava
...,...,...,...
99995,3389,"Alfoz de Toro, Zamora",houses_zamora
99996,3390,"Alfoz de Toro, Zamora",houses_zamora
99997,3391,loc_zone,houses_zamora
99998,3392,"Área de Zamora, Zamora",houses_zamora


In [9]:
df_comprobar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   index     100000 non-null  int64 
 1   loc_zone  100000 non-null  object
 2   source    100000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [10]:
df_c2=df_comprobar.groupby(['loc_zone','source'])['index'].count().to_frame()
df_c2

,,index
loc_zone,source,
"A Barcala, A Coruña",houses_coruna,78
A Coruña,houses_coruna,292
"Alfoz de Toro, Zamora",houses_zamora,179
"Aliste, Zamora",houses_zamora,56
"Aljarafe, Sevilla",houses_sevilla,1655
...,...,...
"Área de Sayago, Zamora",houses_zamora,120
"Área de Segovia, Segovia",houses_segovia,613
"Área de Sepúlveda - Cantalejo, Segovia",houses_segovia,459


In [11]:
df_comprobar.to_csv('df_comprobar.csv', index=False)

In [12]:
df_c2.to_csv('df_c2.csv', index=True)

In [13]:
df_c2.to_csv('df_c3.csv', index=True)

In [14]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 3393
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ad_description                 95426 non-null   object 
 1   ad_last_update                 100000 non-null  object 
 2   air_conditioner                100000 non-null  object 
 3   balcony                        100000 non-null  object 
 4   bath_num                       100000 non-null  object 
 5   built_in_wardrobe              100000 non-null  object 
 6   chimney                        100000 non-null  object 
 7   condition                      86059 non-null   object 
 8   construct_date                 32059 non-null   object 
 9   energetic_certif               74691 non-null   object 
 10  floor                          79693 non-null   object 
 11  garage                         40811 non-null   object 
 12  garden                         1

In [15]:
for i in range(len(result)):
    if (result.iloc[i,24]=='Aljarafe, Sevilla') and (result.iloc[i,36]=='houses_valladolid'):
        result.iloc[i,36]='houses_sevilla'
    if (result.iloc[i,24]=='Área Metropolitana, Sevilla') and (result.iloc[i,36]=='houses_valladolid'):
        result.iloc[i,36]='houses_sevilla'
        #print("sustituido ", i,result.iloc[i,24], result.iloc[i,36])
    if (result.iloc[i,24]=='Sevilla capital, Sevilla') and (result.iloc[i,36]=='houses_valladolid'):
        result.iloc[i,36]='houses_sevilla'
        #print("sustituido ", i,result.iloc[i,24], result.iloc[i,36])
    if (result.iloc[i,24]=='Tarragonès, Tarragona') and (result.iloc[i,36]=='houses_valladolid'):
        result.iloc[i,36]='houses_tarragona'
        #print("sustituido ", i,result.iloc[i,24], result.iloc[i,36])
    if (result.iloc[i,24]=='Álava') and (result.iloc[i,36]=='houses_bizkaia'):
        result.iloc[i,36]='houses_alava'
        #print("sustituido ", i,result.iloc[i,24], result.iloc[i,36])

In [16]:
df_c2=result.groupby(['loc_zone','source'])['house_id'].count().to_frame()
df_c2

,,house_id
loc_zone,source,
"A Barcala, A Coruña",houses_coruna,78
A Coruña,houses_coruna,292
"Alfoz de Toro, Zamora",houses_zamora,179
"Aliste, Zamora",houses_zamora,56
"Aljarafe, Sevilla",houses_sevilla,2525
...,...,...
"Área de Sayago, Zamora",houses_zamora,120
"Área de Segovia, Segovia",houses_segovia,613
"Área de Sepúlveda - Cantalejo, Segovia",houses_segovia,459


In [17]:
result.loc_zone.unique()

array(['Zuya, Álava', 'Álava', 'Salvatierra, Álava',
       'Laguardia-Rioja Alavesa, Álava',
       'Campezo- Montaña Alavesa, Álava', 'Ayala, Álava', 'Añana, Álava',
       'Campo de Hellín, Albacete', 'loc_zone',
       'Sierra de Alcaraz-Segura, Albacete', 'La Manchuela, Albacete',
       'La Mancha, Albacete', 'Corredor de Almansa, Albacete',
       'Centro, Albacete', 'Alt Vinalopó, Alicante',
       'Vinalopó Mitjà, Alicante', 'Marina Baixa, Alicante',
       'Ibiza, Balears (Illes)', 'Menorca, Balears (Illes)',
       'Balears (Illes)', 'Mallorca, Balears (Illes)',
       'Alt Penedès, Barcelona', 'Encartaciones, Vizcaya',
       'Uribe-Butroe, Vizcaya', 'Lea-Artibai, Vizcaya',
       'Duranguesado, Vizcaya', 'Gran Bilbao, Vizcaya',
       'Busturialdea, Vizcaya', 'Arratia-Nervión, Vizcaya',
       'Sierra de Cádiz, Cádiz', 'Montiel, Ciudad Real', 'Ciudad Real',
       'Los Montes, Ciudad Real', 'La Mancha, Ciudad Real',
       'Calatrava, Ciudad Real', 'Xallas, A Coruña',
    

In [18]:
result['provincia']= result['source'].replace({'houses_':''}, regex=True)

In [19]:
#del result['source']
#result.to_csv('spanish_houses.csv', index=False)

In [20]:
result.provincia

0        alava
1        alava
2        alava
3        alava
4        alava
         ...  
3389    zamora
3390    zamora
3391    zamora
3392    zamora
3393    zamora
Name: provincia, Length: 100000, dtype: object

In [21]:
result.shape

(100000, 43)

In [22]:
result

,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,energetic_certif,...,swimming_pool,terrace,unfurnished,source,number_of_companies_prov,population_prov,companies_prov_vs_national_%,population_prov_vs_national_%,renta_media_prov,provincia
0,Precio chalet individual en la localidad de Ab...,Anuncio actualizado el 27 de marzo,0,0,2,0,0,segunda mano/buen estado,NaN,NaN,...,0,1,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,alava
1,"Atico de 80m2, para entrar a vivir, con salón ...",más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,2006.0,no indicado,...,0,0,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,alava
2,B/ Etxaguen. Casa de reciente construcción con...,más de 5 meses sin actualizar,0,0,3,0,0,segunda mano/buen estado,NaN,no indicado,...,0,1,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,alava
3,Se vende vivienda en abornikano (ayuntamiento ...,más de 5 meses sin actualizar,0,1,1,1,1,segunda mano/buen estado,NaN,en trámite,...,0,1,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,alava
4,Negociables.,más de 5 meses sin actualizar,0,0,1,0,0,segunda mano/buen estado,NaN,no indicado,...,1,1,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,alava
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3389,casa de pueblo con patio.,Anuncio actualizado el 29 de enero,0,0,1,0,0,segunda mano/para reformar,NaN,no indicado,...,0,0,NaN,houses_zamora,11632,174549,0.35,0.37,NaN,zamora
3390,"Superf. 2299 m², 1090 m² solar, 10 habitacione...",Anuncio actualizado el 27 de septiembre,0,0,2,0,0,NaN,1908,no indicado,...,0,0,NaN,houses_zamora,11632,174549,0.35,0.37,NaN,zamora
3391,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,energetic_certif,...,swimming_pool,terrace,unfurnished,houses_zamora,11632,174549,0.35,0.37,NaN,zamora
3392,Inmueble de 89 m² ubicado en ZAMORA (Zamora).,Anuncio actualizado el 10 de abril,0,0,2,0,0,NaN,NaN,no indicado,...,0,0,NaN,houses_zamora,11632,174549,0.35,0.37,NaN,zamora


In [23]:
data=result[["provincia", "renta_media_prov"]]

In [24]:
data.groupby(['provincia'])['renta_media_prov'].sum().to_frame()

,renta_media_prov
provincia,
alava,7.811533e+07
albacete,0.000000e+00
alicante,4.205890e+04
balears,0.000000e+00
barcelona,2.523489e+04
bizkaia,4.573004e+08
cadiz,0.000000e+00
ciudad_real,0.000000e+00
coruna,1.901998e+05


In [25]:
data.provincia.unique()

array(['alava', 'albacete', 'alicante', 'balears', 'barcelona', 'bizkaia',
       'cadiz', 'ciudad_real', 'coruna', 'gipuzkoa', 'girona', 'huelva',
       'madrid', 'santa_cruz_de_tenerife', 'segovia', 'sevilla', 'soria',
       'valencia', 'valladolid', 'tarragona', 'zamora'], dtype=object)

In [26]:
result.provincia.unique()

array(['alava', 'albacete', 'alicante', 'balears', 'barcelona', 'bizkaia',
       'cadiz', 'ciudad_real', 'coruna', 'gipuzkoa', 'girona', 'huelva',
       'madrid', 'santa_cruz_de_tenerife', 'segovia', 'sevilla', 'soria',
       'valencia', 'valladolid', 'tarragona', 'zamora'], dtype=object)

In [27]:
valor_catastral = pd.read_csv(r'C:\Users\hecto\OneDrive\Documentos\IT Data Science\proyecto\idealista\valor_catastral_medio.csv',on_bad_lines='skip')

In [28]:
valor_catastral

,provincia,Valor_catastral_medio,CC_AA,CC_AA_no
0,alava,80334,Pais_Vasco,9
1,albacete,55341,Castilla_La_Mancha,4
2,alicante,50561,Valencia,10
3,balears,78362,Balears,2
4,barcelona,86963,Catalunya,6
5,bizkaia,59523,Pais_Vasco,9
6,cadiz,61089,Andalucia,1
7,ciudad_real,50228,Castilla_La_Mancha,4
8,coruna,53760,Galicia,7
9,gipuzkoa,124406,Pais_Vasco,9


In [29]:
valor_catastral.provincia.unique()

array(['alava', 'albacete', 'alicante', 'balears', 'barcelona', 'bizkaia',
       'cadiz', 'ciudad_real', 'coruna', 'gipuzkoa', 'girona', 'huelva',
       'madrid', 'santa_cruz_de_tenerife', 'segovia', 'sevilla', 'soria',
       'tarragona', 'valencia', 'valladolid', 'zamora'], dtype=object)

In [30]:
data_join=result.set_index('provincia').join(valor_catastral.set_index('provincia'))

In [31]:
result= data_join.reset_index()

In [32]:
result

,provincia,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,...,unfurnished,source,number_of_companies_prov,population_prov,companies_prov_vs_national_%,population_prov_vs_national_%,renta_media_prov,Valor_catastral_medio,CC_AA,CC_AA_no
0,alava,Precio chalet individual en la localidad de Ab...,Anuncio actualizado el 27 de marzo,0,0,2,0,0,segunda mano/buen estado,NaN,...,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,80334,Pais_Vasco,9
1,alava,"Atico de 80m2, para entrar a vivir, con salón ...",más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,2006.0,...,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,80334,Pais_Vasco,9
2,alava,B/ Etxaguen. Casa de reciente construcción con...,más de 5 meses sin actualizar,0,0,3,0,0,segunda mano/buen estado,NaN,...,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,80334,Pais_Vasco,9
3,alava,Se vende vivienda en abornikano (ayuntamiento ...,más de 5 meses sin actualizar,0,1,1,1,1,segunda mano/buen estado,NaN,...,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,80334,Pais_Vasco,9
4,alava,Negociables.,más de 5 meses sin actualizar,0,0,1,0,0,segunda mano/buen estado,NaN,...,NaN,houses_alava,19147,328868,0.57,0.70,19889.0,80334,Pais_Vasco,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,zamora,casa de pueblo con patio.,Anuncio actualizado el 29 de enero,0,0,1,0,0,segunda mano/para reformar,NaN,...,NaN,houses_zamora,11632,174549,0.35,0.37,NaN,45595,Castilla_Leon,5
99996,zamora,"Superf. 2299 m², 1090 m² solar, 10 habitacione...",Anuncio actualizado el 27 de septiembre,0,0,2,0,0,NaN,1908,...,NaN,houses_zamora,11632,174549,0.35,0.37,NaN,45595,Castilla_Leon,5
99997,zamora,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,...,unfurnished,houses_zamora,11632,174549,0.35,0.37,NaN,45595,Castilla_Leon,5
99998,zamora,Inmueble de 89 m² ubicado en ZAMORA (Zamora).,Anuncio actualizado el 10 de abril,0,0,2,0,0,NaN,NaN,...,NaN,houses_zamora,11632,174549,0.35,0.37,NaN,45595,Castilla_Leon,5


In [33]:
result=result.drop(columns = ['ground_size','renta_media_prov','companies_prov_vs_national_%','population_prov_vs_national_%'])
result.head()

,provincia,ad_description,ad_last_update,air_conditioner,balcony,bath_num,built_in_wardrobe,chimney,condition,construct_date,...,storage_room,swimming_pool,terrace,unfurnished,source,number_of_companies_prov,population_prov,Valor_catastral_medio,CC_AA,CC_AA_no
0,alava,Precio chalet individual en la localidad de Ab...,Anuncio actualizado el 27 de marzo,0,0,2,0,0,segunda mano/buen estado,NaN,...,0,0,1,NaN,houses_alava,19147,328868,80334,Pais_Vasco,9
1,alava,"Atico de 80m2, para entrar a vivir, con salón ...",más de 5 meses sin actualizar,0,0,2,0,0,segunda mano/buen estado,2006.0,...,1,0,0,NaN,houses_alava,19147,328868,80334,Pais_Vasco,9
2,alava,B/ Etxaguen. Casa de reciente construcción con...,más de 5 meses sin actualizar,0,0,3,0,0,segunda mano/buen estado,NaN,...,1,0,1,NaN,houses_alava,19147,328868,80334,Pais_Vasco,9
3,alava,Se vende vivienda en abornikano (ayuntamiento ...,más de 5 meses sin actualizar,0,1,1,1,1,segunda mano/buen estado,NaN,...,1,0,1,NaN,houses_alava,19147,328868,80334,Pais_Vasco,9
4,alava,Negociables.,más de 5 meses sin actualizar,0,0,1,0,0,segunda mano/buen estado,NaN,...,1,1,1,NaN,houses_alava,19147,328868,80334,Pais_Vasco,9


In [34]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   provincia                 100000 non-null  object
 1   ad_description            95426 non-null   object
 2   ad_last_update            100000 non-null  object
 3   air_conditioner           100000 non-null  object
 4   balcony                   100000 non-null  object
 5   bath_num                  100000 non-null  object
 6   built_in_wardrobe         100000 non-null  object
 7   chimney                   100000 non-null  object
 8   condition                 86059 non-null   object
 9   construct_date            32059 non-null   object
 10  energetic_certif          74691 non-null   object
 11  floor                     79693 non-null   object
 12  garage                    40811 non-null   object
 13  garden                    100000 non-null  object
 14  heati

In [35]:
result.to_csv('spanish_houses.csv', index=False)

In [36]:
result.provincia.unique()

array(['alava', 'albacete', 'alicante', 'balears', 'barcelona', 'bizkaia',
       'cadiz', 'ciudad_real', 'coruna', 'gipuzkoa', 'girona', 'huelva',
       'madrid', 'santa_cruz_de_tenerife', 'segovia', 'sevilla', 'soria',
       'tarragona', 'valencia', 'valladolid', 'zamora'], dtype=object)

In [37]:
df_c4=result.groupby(['provincia'])['Valor_catastral_medio'].count().to_frame()
df_c4

,Valor_catastral_medio
provincia,
alava,3928
albacete,4469
alicante,1946
balears,24880
barcelona,885
bizkaia,23075
cadiz,787
ciudad_real,1819
coruna,8341


In [38]:
result.provincia.shape

(100000,)

In [39]:
result.CC_AA.unique()

array(['Pais_Vasco', 'Castilla_La_Mancha', 'Valencia', 'Balears',
       'Catalunya', 'Andalucia', 'Galicia', 'Madrid', 'Canarias',
       'Castilla_Leon'], dtype=object)

In [40]:
result.CC_AA.shape

(100000,)

In [41]:
df_c3=result.groupby(['CC_AA','provincia'])['CC_AA'].count().to_frame()
df_c3

CC_AA
CC_AA              provincia                    
Andalucia          cadiz                     787
                   huelva                    658
                   sevilla                  5082
Balears            balears                 24880
Canarias           santa_cruz_de_tenerife   1222
Castilla_La_Mancha albacete                 4469
                   ciudad_real              1819
Castilla_Leon      segovia                  1529
                   soria                    1373
                   valladolid                218
                   zamora                   3394
Catalunya          barcelona                 885
                   girona                   1811
                   tarragona                 219
Galicia            coruna                   8341
Madrid             madrid                   5255
Pais_Vasco         alava                    3928
                   bizkaia                 23075
                   gipuzkoa                 7705
Valencia           alicante                 1946
                   valencia                 1404

In [42]:
df_c3.shape

(21, 1)